In [14]:
import os.path
import sqlite3
import socket
import numpy as np


from torch.utils.data import Dataset

if socket.gethostname() == 'LTSSL-sKTPpP5Xl':
    data_dir = 'C:\\Users\\ams90\\PycharmProjects\\ConceptsBirds\\data'
elif socket.gethostname() == 'LAPTOP-NA88OLS1':
    data_dir = 'D:\\data\\caltecBirds\\CUB_200_2011'
elif socket.gethostname() == 'andrew-ubuntu':
    data_dir ='/home/andrew/Data/CUB_200_2011'
else:
    data_dir = '/home/bwc/ams90/datasets/caltecBirds/CUB_200_2011'


In [15]:
conn = sqlite3.connect(database=os.path.join(data_dir, 'birds.db'))
conn.create_function("log", 1, np.log)
conn.create_function("exp", 1, np.exp)
cursor = conn.cursor()

In [16]:
classes      = {class_id: class_name     for (class_id, class_name)     in cursor.execute("select class_id, class_name from classes").fetchall()}
concepts     = {concept_id: concept_name for (concept_id, concept_name) in cursor.execute("select concept_id,concept_name from concepts").fetchall()}
attributes   = {attribute_id: attribute_id for (attribute_id, ) in cursor.execute("select attribute_id from attributes").fetchall()}
class_counts = {class_id: prior for (class_id, prior) in cursor.execute("select class_id, class_prior from nb_class_counts where trainset = 1").fetchall()}


In [17]:
count, correct, unknown, wrong = 0, 0, 0, 0
loop = conn.cursor()

count, correct = 0, [0,0,0]
for image_id, class_id in loop.execute("select image_id, class_id from images where trainset = 1").fetchall():
    
    present_attribute_ids = [row[0] for row in cursor.execute("select attribute_id from image_attributes where present = 1 and image_id = ?", (image_id,)).fetchall()]
    
    class_log_probabilities = [row for row in cursor.execute("""select  sum(p.naive_test_probability),
                                                                        sum(p.naive_train_probability),
                                                                        sum(p.probability)
                                                                 from   class_attribute_probabilities p,
                                                                       image_attributes ia
                                                                 where  p.attribute_id = ia.attribute_id
                                                                 and    ia.present     = 1 
                                                                 and    ia.image_id    = ?
                                                                 group by p.class_id
                                                                 order by p.class_id """, (image_id,)).fetchall()]
    
    class_probabilities = np.exp(class_log_probabilities)
    class_probabilities = [[field for field in row] for row in class_probabilities]
    class_probabilities = [[row[column] for row in class_probabilities] for column in range(3)]
    

    for classifier in range(3):
    
        total_prob = sum(class_probabilities[classifier])       
        if total_prob > 0:
            norm_probabilities  = [probability / total_prob for probability in class_probabilities[classifier]]
            max_prob = max(norm_probabilities)
            pred_class_id = norm_probabilities.index(max_prob) + 1
            if pred_class_id == class_id: correct[classifier] += 1
       
    
    count += 1

print(image_id, class_id, correct, [round(corr/count,4) for corr in correct])
    

loop.close()




11787 200 [1786, 4229, 3800] [0.298, 0.7055, 0.634]


In [18]:

cursor = conn.cursor()

class_attribute_probabilities = cursor.execute(
    """ select cls.class_id,
               cls.class_name,
               cap.attribute_id,
               exp(cap.probability),
               exp(cap.naive_train_probability),
               exp(naive_test_probability)
         from  classes cls,
               class_attribute_probabilities cap
         where  cls.class_id = 1
         and    cap.attribute_id in (194, 195)
         and   cls.class_id = cap.class_id

    """).fetchall()

for record in class_attribute_probabilities:
    print(record)

cursor.close()


(1, '001.Black_footed_Albatross', 194, 0.18181818181799997, 0.16666666666666669, 0.13333333333333333)
(1, '001.Black_footed_Albatross', 195, 0.05303030303029999, 0.06666666666666667, 0.03333333333333333)
